<a href="https://colab.research.google.com/github/sjabeen/UMD-FIRE-QML/blob/master/qkd_e91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quantum Key Distribution using entangled pairs

The E91 protocol developed by Artur Ekert in 1991 is based on the use of entangled states and Bell's theorem. Two electrons A and B can be prepared in states that they can not be considered separately from each other. One of these states is the singlet state where the spins of the electrons face opposite directions.
$$|\psi\rangle_s=\frac{1}{\sqrt{2}}( |\uparrow\rangle_A \otimes |\downarrow\rangle_B - |\downarrow\rangle_A \otimes |\uparrow\rangle_B )= \frac{1}{\sqrt{2}} (|\uparrow\downarrow\rangle - |\downarrow\uparrow\rangle)$$
The singlet state is asymmetric with exchange of both electrons and the total spin is zero.

In [ ]:
import os, time, random
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.providers.jobstatus import JobStatus
try:
    from qiskit_ionq import IonQProvider
except (ImportError):
    print("Installing qiskit_ionq ...")
    !pip install --quiet cirq_ionq
    from qiskit_ionq import IonQProvider

#get access to IonQ provider (token required)
try:
    ionq_token=os.getenv('IONQ_API_TOKEN')
except:
except:
    ionq_token = "unknown"  # <<<< replace "unknown" with your IonQ API token!!!
    if ionq_token == "unknown":
        ionq_token = input("IonQ API token:")
provider = IonQProvider(ionq_token)

# create backends for simulator and hardware
aer_sim = Aer.get_backend("aer_simulator")
ionq_sim = provider.get_backend("ionq_simulator")
ionq_qpu = provider.get_backend("ionq_qpu")

########################
#first a simple function to draw the circuit and run it nshots times
#  (for IonQ hardware wait up to max_wait_minutes to get the job processed in the queue)

def draw_and_run(qc, backend=aer_sim, nshots=1000, max_wait_minutes=10):
    display(qc.draw('mpl'))
    job = backend.run(qc, shots=nshots)
    # jobs are either run locally (AER) or submitted to IonQ
    # if it takes too long get the job done on IonQ hardware, stop it
    if backend == ionq_qpu:
        while job.status() is not JobStatus.DONE:
            time.sleep(60)
            print("Job status is", job.status())
            if job.status() is JobStatus.ERROR:
                print("Error running job on IonQ hardware")
                return
            max_wait_minutes -= 1
            if max_wait_minutes == 0:
                print("It takes too long to get job started on IonQ hardware")
                return

    cnts = job.result().get_counts()
    allstates = []
    for j in range(2**qc.num_qubits):
        allstates.append(bin(j)[2:].zfill(qc.num_qubits))
    for j in allstates:
        if j not in cnts.keys():
            cnts[j] = 0
    counts = {k:cnts[k] for k in allstates}
    display(plot_histogram(counts))

The singlet state is realized by this Bell state:

In [ ]:
#antisymmetric singlet state:
qc = QuantumCircuit(2,2)
qc.x([0,1])
qc.h(0)
qc.cx(0,1)
qc.measure([0,1],[0,1])
draw_and_run(qc, ionq_sim)

To implement the E91 quantum key distribution protocol, there must be a source of qubits prepared in the singlet state. It does not matter to whom this source belongs: to Alice, to Bob, to a trusted third-party Charlie or even to eavesdropper Eve. One qubit is (sent to and) measured by Alice, the other (sent to and) measured by Bob.

Each party chooses a random event-by-event basis for the measurement of the spin projection of her/his qubit in the following directions:

$$ \mbox{Alice:} \qquad \qquad \vec{a}_1=\left(\begin{array}{c}1 \\ 0 \\ 0\end{array}\right) \quad \mbox{(X observable)} ;
\qquad \vec{a}_2=\frac{1}{\sqrt{2}}\left(\begin{array}{c}1 \\ 0 \\ 1\end{array}\right) \quad \mbox{(W observable)} ;
\qquad \vec{a}_3=\left(\begin{array}{c}0 \\ 0 \\ 1\end{array}\right) \quad \mbox{(Z observable)} $$

$$ \mbox{Bob:} \qquad \vec{b}_1=\frac{1}{\sqrt{2}}\left(\begin{array}{c}1 \\ 0 \\ 1\end{array}\right) \quad \mbox{(W observable)} ;
\qquad \vec{b}_2=\left(\begin{array}{c}0 \\ 0 \\ 1\end{array}\right) \quad \mbox{(Z observable)} ;
\qquad \vec{b}_3=\frac{1}{\sqrt{2}}\left(\begin{array}{c}-1 \\ 0 \\ 1\end{array}\right) \quad \mbox{(V observable)} $$
<div>
<img src="attachment:Screen%20Shot%202022-10-29%20at%204.17.29%20PM.png" width="400">
</div>
Any time that Alice measured in $\vec{a}_3$ direction and Bob measured in $\vec{b}_2$ direction, they must get the opposite projection result; same for Alice measuring along $\vec{a}_2$ and Bob measuring along $\vec{b}_1$.

In [ ]:
# Creating registers (2 additional classical registers to include Eve's measurements)
qr = QuantumRegister(2, name="qr")
cr = ClassicalRegister(4, name="cr")

####################
# Charlie's device creating a singlet state
singlet = QuantumCircuit(qr, cr)
singlet.x([0,1])
singlet.h(0)
singlet.cx(0,1)

###################
# Alice's measurement circuits

# measure the spin projection of Alice's qubit onto the a_1 direction (X basis)
measureA1 = QuantumCircuit(qr, cr, name='measureA1')
measureA1.h(qr[0])
measureA1.measure(qr[0],cr[0])

# measure the spin projection of Alice's qubit onto the a_2 direction (W basis)
measureA2 = QuantumCircuit(qr, cr, name='measureA2')
measureA2.s(qr[0])
measureA2.h(qr[0])
measureA2.t(qr[0])
measureA2.h(qr[0])
measureA2.measure(qr[0],cr[0])

# measure the spin projection of Alice's qubit onto the a_3 direction (standard Z basis)
measureA3 = QuantumCircuit(qr, cr, name='measureA3')
measureA3.measure(qr[0],cr[0])

###########
# Bob's measurement circuits

# measure the spin projection of Bob's qubit onto the b_1 direction (W basis)
measureB1 = QuantumCircuit(qr, cr, name='measureB1')
measureB1.s(qr[1])
measureB1.h(qr[1])
measureB1.t(qr[1])
measureB1.h(qr[1])
measureB1.measure(qr[1],cr[1])

# measure the spin projection of Bob's qubit onto the b_2 direction (standard Z basis)
measureB2 = QuantumCircuit(qr, cr, name='measureB2')
measureB2.measure(qr[1],cr[1])

# measure the spin projection of Bob's qubit onto the b_3 direction (V basis)
measureB3 = QuantumCircuit(qr, cr, name='measureB3')
measureB3.s(qr[1])
measureB3.h(qr[1])
measureB3.tdg(qr[1])
measureB3.h(qr[1])
measureB3.measure(qr[1],cr[1])

## Lists of measurement circuits
aliceMeasurements = [measureA1, measureA2, measureA3]
bobMeasurements = [measureB1, measureB2, measureB3]


Suppose Alice and Bob want to generate a secret key using N singlet states prepared by Charlie.<br>
They must choose the directions onto which they will measure the spin projections of their qubits. To do this, Alice and Bob create 'bases' lists with randomly generated elements.

In [ ]:
nSinglets = 30
basesA = [random.randint(1, 3) for i in range(nSinglets)]
basesB = [random.randint(1, 3) for i in range(nSinglets)]
#combine Charlie's device and Alice's and Bob's detectors into one circuit
circuits=[]
for j in range(nSinglets):
    # create the name of the j-th circuit depending on Alice's and Bob's measurement choices
    circName = 'Circ' + str(j) + '_A' + str(basesA[j]) + '_B' + str(basesB[j])

    # create the joint measurement circuit
    # add Alice's and Bob's measurement circuits to the singlet state curcuit
    circ = singlet.copy(name=circName)
    circ = circ.compose(aliceMeasurements[basesA[j]-1])
    circ = circ.compose(bobMeasurements[basesB[j]-1])

    # add the created circuit to the circuits list
    circuits.append(circ)
print("Created {} circuits".format(len(circuits)))

In [ ]:
for j in [0, -1]:
    print("Circuit {}:".format(circuits[j].name))
    display(draw_and_run(circuits[j]))

In [ ]:
# select backend and execute circuits
# function returns list of measurement results
def run_jobs(qcircuits, backend=aer_sim, max_wait_minutes=10):
    joblist = []
    outcome = []
    for qc in qcircuits:
        joblist.append(backend.run(qc, shots=1))

    # get the results
    for job in joblist:
        # jobs are either run locally (AER) or submitted to IonQ
        # if it takes too long get the job done on IonQ hardware, stop it
        if backend == ionq_qpu:
            while job.status() is not JobStatus.DONE:
                time.sleep(60)
                print("Job status is", job.status())
                if job.status() is JobStatus.ERROR:
                    print("Error running job on IonQ hardware")
                    return outcome
                max_wait_minutes -= 1
                if max_wait_minutes == 0:
                    print("It takes too long to get job started on IonQ hardware")
                    return outcome
        res = job.result().get_counts()
        outcome.append(list(res.keys())[0] if isinstance(res, dict) else str(res))
    return outcome

In [ ]:
results = run_jobs(circuits)

aliceResults = []
bobResults = []
for res in results:
    resB,resA = res[-2:-1],res[-1:]
    #print(res,resA,resB)
    bobResults.append(1 if resB == '1' else -1)
    aliceResults.append(1 if resA == '1' else -1)

if nSinglets < 65:
    print("Alice's results:",aliceResults)
    print("Bob's results:  ",bobResults)

In [ ]:
aliceKey = [] # Alice's key string k
bobKey = [] # Bob's key string k'

# comparing the stings with measurement choices
for i in range(len(results)):
    # if Alice and Bob have measured the spin projections onto the a_2,b_1 or a_3,b_2 directions
    if (basesA[i] == 2 and basesB[i] == 1) or (basesA[i] == 3 and basesB[i] == 2):
        aliceKey.append(aliceResults[i]) # record the i-th result obtained by Alice as the bit of the secret key k
        bobKey.append(-bobResults[i]) # record the multiplied by -1 i-th result obtained Bob as the bit of the secret key k'

# length of the secret key
keyLength = len(aliceKey)
# number of mismatching bits in Alice's and Bob's keys
KeyMismatches = 0
for j in range(keyLength):
    if aliceKey[j] != bobKey[j]:
        KeyMismatches += 1

<h3>The CHSH correlation value</h3>

Alice and Bob want to be sure that there was no interference in the communication session. To do that, they calculate the CHSH correlation value using the results obtained after the measurements of spin projections
onto the $(\vec{a}_1,\vec{b}_1), (\vec{a}_1,\vec{b}_3), (\vec{a}_3,\vec{b}_1)$ and $(\vec{a}_3,\vec{b}_3)$ directions.
This is equivalent to the measurement of the observables $X\otimes W, X\otimes V, Z\otimes W, Z\otimes V$, resp.

The expectation value for the joint measurement of the observables A and B is given by:
$$\langle A\otimes B\rangle_\psi =\sum_{j,k} a_j b_k P(A\models a_j, B\models b_k) = \sum_{j,k} a_j b_k P_\psi(a_j,b_k) , $$
where $P_\psi(A\models a_j)$ is the probability of obtainig the result $a_j$ when measuring the observable A for the state $|\psi\rangle$.

Here, A and B are spin projection observables, so the corresponding eigenvalues are $a_j,b_k=\pm 1$,
to the expectation value becomes
$$\langle A(a_j)\otimes B(b_k)\rangle_\psi = P(-1,-1) - P(-1,1) - P(1,-1) + P(1,1)$$






In [ ]:
# function that calculates CHSH correlation value
def chsh_corr(results):

    # lists with the counts of measurement results
    # each element represents the number of (-1,-1), (-1,1), (1,-1) and (1,1) results respectively
    patterns = ['00','01','10','11']
    countA1B1 = [0, 0, 0, 0] # XW observable
    countA1B3 = [0, 0, 0, 0] # XV observable
    countA3B1 = [0, 0, 0, 0] # ZW observable
    countA3B3 = [0, 0, 0, 0] # ZV observable

    for i,res in enumerate(results):
        resBA = res[-2:]
        ind = patterns.index(resBA)
        # if the spins of the qubits of the i-th singlet were projected onto the a_1/b_1 directions
        if (basesA[i] == 1 and basesB[i] == 1):
            countA1B1[ind] += 1
        elif (basesA[i] == 1 and basesB[i] == 3):
            countA1B3[ind] += 1
        elif (basesA[i] == 3 and basesB[i] == 1):
            countA3B1[ind] += 1
        # if the spins of the qubits of the i-th singlet were projected onto the a_3/b_3 directions
        elif (basesA[i] == 3 and basesB[i] == 3):
            countA3B3[ind] += 1

    # expectation values of XW, XV, ZW and ZV observables
    expect11 = (countA1B1[0] - countA1B1[1] - countA1B1[2] + countA1B1[3])/sum(countA1B1) # -1/sqrt(2)
    expect13 = (countA1B3[0] - countA1B3[1] - countA1B3[2] + countA1B3[3])/sum(countA1B3) # 1/sqrt(2)
    expect31 = (countA3B1[0] - countA3B1[1] - countA3B1[2] + countA3B1[3])/sum(countA3B1) # -1/sqrt(2)
    expect33 = (countA3B3[0] - countA3B3[1] - countA3B3[2] + countA3B3[3])/sum(countA3B3) # -1/sqrt(2)

    corr = expect11 - expect13 - expect31 + expect33 # calculate the CHSC correlation value (3)

    return corr

In [ ]:
corr = chsh_corr(results) # CHSH correlation value

# CHSH inequality test
print('CHSH correlation value: {}  (expected: -{})\n'.format(round(corr,3), round(2*np.sqrt(2),3)))

# Keys
print('Length of the key: ' + str(keyLength))
print('Number of mismatching bits: ' + str(KeyMismatches))

<h2>E91 with interception</h2>

In [ ]:
# measurement of the spin projection of Alice's qubit onto the a_2 direction (W basis)
measureEA2 = QuantumCircuit(qr, cr, name='measureEA2')
measureEA2.s(qr[0])
measureEA2.h(qr[0])
measureEA2.t(qr[0])
measureEA2.h(qr[0])
measureEA2.measure(qr[0],cr[2])

# measurement of the spin projection of Allice's qubit onto the a_3 direction (standard Z basis)
measureEA3 = QuantumCircuit(qr, cr, name='measureEA3')
measureEA3.measure(qr[0],cr[2])

# measurement of the spin projection of Bob's qubit onto the b_1 direction (W basis)
measureEB1 = QuantumCircuit(qr, cr, name='measureEB1')
measureEB1.s(qr[1])
measureEB1.h(qr[1])
measureEB1.t(qr[1])
measureEB1.h(qr[1])
measureEB1.measure(qr[1],cr[3])

# measurement of the spin projection of Bob's qubit onto the b_2 direction (standard Z measurement)
measureEB2 = QuantumCircuit(qr, cr, name='measureEB2')
measureEB2.measure(qr[1],cr[3])

# lists of measurement circuits
eveMeasurements = [measureEA2, measureEA3, measureEB1, measureEB2]

In [ ]:
# list of Eve's measurement choices
# 2 elements per row for Eve's measurement of Alice's and Bob's qubits
basesE2 = []

for j in range(nSinglets):
    if random.uniform(0, 1) < 0.5:  # in 50% of cases perform the WW measurement
        basesE2.append([0, 2])
    else:                           # in 50% of cases perform the ZZ measurement
        basesE2.append([1, 3])

In [ ]:
circuits = [] # the list in which the created circuits will be stored

for j in range(nSinglets):
    # create the name of the j-th circuit depending on Alice's, Bob's and Eve's choices of measurement
    circName = 'Circ'+str(j)+'_A' + str(basesA[j]) + '_B' + str(basesB[j]) + '_E' + str(basesE2[j][0]+1) + str(basesE2[j][1]+1)

    # create the joint measurement circuit
    # add Alice's and Bob's measurement circuits to the singlet state curcuit
    circ = singlet.copy(name=circName)
    circ = circ.compose(eveMeasurements[basesE2[j][0]])
    circ = circ.compose(eveMeasurements[basesE2[j][1]])
    circ = circ.compose(aliceMeasurements[basesA[j]-1])
    circ = circ.compose(bobMeasurements[basesB[j]-1])
    # add the created circuit to the circuits list
    circuits.append(circ)

In [ ]:
#results = run_jobs(circuits)

aliceResults = []
bobResults = []
# list of Eve's results: 1st column from measurements of Alice's qubits, 2nd column from Bob's qubits
eveResults = []

# recording the measurement results
patterns = ['00','01','10','11']
EveValues = [[-1,-1],[-1,1],[1,-1],[1,1]]
for res in results:
    resE,resB,resA = res[0:2],res[-2:-1],res[-1:]
    #print(res,resE,resA,resB)
    bobResults.append(1 if resB == '1' else -1)
    aliceResults.append(1 if resA == '1' else -1)
    # Eve
    ind = patterns.index(resE)
    eveResults.append(EveValues[ind])
print(aliceResults)
print(bobResults)
print(eveResults)

In [ ]:
aliceKey = [] # Alice's key string a
bobKey = [] # Bob's key string a'
eveKeys = [] # Eve's keys; the 1-st column is the key of Alice, and the 2-nd is the key of Bob

# comparing results with measurement choices
for j in range(nSinglets):
    # if Alice and Bob measured the spin projections onto the a_2,b_1 or a_3,b_2 directions
    if (basesA[j] == 2 and basesB[j] == 1) or (basesA[j] == 3 and basesB[j] == 2):
        aliceKey.append(aliceResults[j]) # record the j-th result obtained by Alice as bit of her secret key
        bobKey.append(-bobResults[j]) # record Bob's j-th result (multiplied by -1) as bit of his secret key
        eveKeys.append([eveResults[j][0], -eveResults[j][1]]) # record the i-th bits of the keys of Eve

keyLength = len(aliceKey) # length of the secret key

KeyMismatches = 0 # number of mismatching bits in the keys of Alice and Bob
EveAKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Alice
EveBKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Bob

for j in range(keyLength):
    if aliceKey[j] != bobKey[j]:
        KeyMismatches += 1
    if eveKeys[j][0] != aliceKey[j]:
        EveAKeyMismatches += 1
    if eveKeys[j][1] != bobKey[j]:
        EveBKeyMismatches += 1

EveAKeyKnowledge = (keyLength - EveAKeyMismatches)/keyLength # Eve's knowledge of Bob's key
EveBKeyKnowledge = (keyLength - EveBKeyMismatches)/keyLength # Eve's knowledge of Alice's key

In [ ]:
corr = chsh_corr(results)

# CHSH inequality test
print("CHSH correlation value:  {}  (expected: -{})\n".format(round(corr,3), round(2*np.sqrt(2),3)))

# Keys
print('Length of the key: ' + str(keyLength))
print('Number of mismatching bits: ' + str(KeyMismatches) + '\n')

print('Eve\'s knowledge of Alice\'s key: {} %'.format(round(EveAKeyKnowledge *100,2)))
print('Eve\'s knowledge of Bob\'s key:   {} %'.format(round(EveBKeyKnowledge *100,2)))